<a href="https://colab.research.google.com/github/emz95/AIagent/blob/main/Study_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain langgraph langchain-google-genai tavily-python

!pip install -U langchain langchain-community


In [6]:
from typing import TypedDict, List, Literal, Dict

class CurriculumState(TypedDict):
    topic: str
    plan: List[Dict]
    current_week: int
    num_weeks: int
    completed_lessons: List[str]
    quiz_attempts: List[str]
    notes: List[str]
    resources: List[Dict]
    user_feedback: List[str]
    phase: Literal["start", "planning", "resources", "quiz", "leader", "end"]
    phase_history: List[str]


In [7]:
# prompts

def build_study_plan_prompt(topic, num_weeks):
    return f"""
    Create a {num_weeks}-week study plan for the topic: "{topic}".
    Respond ONLY with a valid JSON array containing {num_weeks} objects.

    Each object must have two keys:
    - 'week': (an integer from 1 to {num_weeks})
    - 'focus': (a short, descriptive string summarizing the learning goal for that week)

    Format example:
    [{{"week": 1, "focus": "Intro to {topic}"}}, ..., {{"week": {num_weeks}, "focus": "Advanced topics in {topic}"}}]

    Do NOT include any extra commentary, markdown, or text outside of the JSON array.
    """.strip()


def build_quiz_generation_prompt(topic, current_week, titles):
    formatted_titles = "\n".join(f"- {t}" for t in titles)
    return f"""
    You're an AI tutor generating a short-answer quiz for a student learning about "{topic}".
    Use the following article titles from Week {current_week} to write **3 open-ended questions** that test true understanding of the concepts.

    Guidelines:
    - Each question should focus on one concrete concept or technique from the topics.
    - Ask about definitions, applications, comparisons, or reasoning.
    - Avoid vague questions like "What do you think...?" or "How might...?"
    - No multiple choice. Use only short-answer format.
    - Make sure each question has a clearly correct answer.
    - Do NOT generate questions that:
        - Ask about the meaning of article **titles**
        - Infer themes or implications based only on article names
        - Require speculation or high-level guessing
        - Are vague or abstract

    Format your response like this (valid JSON):

    [
      {{
        "question": "What role do embeddings play in LLMs?",
        "answer": "Embeddings convert text into numerical representations that preserve semantic meaning, allowing LLMs to understand context and relationships between words."
      }},
      ...
    ]

    Here are the article titles:
    {formatted_titles}
    """

def build_grading_prompt(question, expected_answer, student_answer):
    return f"""
    You are grading a student's open-ended response to a quiz question.

    Question: {question}
    Expected Answer: {expected_answer}
    Student's Answer: {student_answer}

    Instructions:
    - Base your judgment only on the student's answer.
    - If they wrote things like "I don't know", "no idea", or left it vague or unrelated, mark it Incorrect.
    - Be strict. Only mark it Correct if they clearly show understanding of the key concept.
    - Use "Partially correct" if they mention related ideas but miss the core.
    - Be sure to state what the correct answer is.
    - Talk as if you are giving feedback directly to the student.

    Reply in **exactly** this format:

    Judgment: Correct | Partially correct | Incorrect
    Explanation: <Briefly explain why and what the correct answer involves>
    """


def build_answer_question_prompt(topic, question):
    return f"""
    You are an AI tutor helping a student learn about "{topic}".
    The student has asked the following question:

    "{question}"

    Instructions:
    - First, determine if the question is clearly related to the topic "{topic}".
    - If it is NOT related, respond: "Sorry, I can only help with questions related to {topic} right now."
    - If it IS related, provide a thoughtful, clear explanation. Use examples or analogies if helpful.
    - Keep the tone friendly, encouraging, and easy to understand.

    Respond as the tutor, directly to the student.
    """

def build_explanation_prompt(current_week, article_titles):
    return f"""
    You are an AI tutor helping a student understand this week's topics.
    Based on the following article titles from Week {current_week}, identify the key topics mentioned and explain them in a clear, student-friendly way.

    Article Titles:
    {chr(10).join(f"- {title}" for title in article_titles)}

    Provide a structured explanation that covers each topic clearly, as if you are teaching it for the first time.
    """


def build_leader_decision_prompt(state, feedback):
    return f"""
    You are a helpful AI learning assistant. Your job is to decide what the student wants to do next
    based on their feedback and the current state of the learning session.

    Here are the phases you can choose from, with descriptions:

    - **planning**: Generate a week-by-week study plan for the selected topic.
    - **resources**: Find and present 2–3 curated articles or videos for the current week's focus.
    - **explain**: Teach the student the core concepts of this week's topic based on the resources found. Only if student wants to regenerate explanation for multiple concepts.
    - **quiz**: Ask 2–3 short-answer questions to check if the student understood this week's material.
    - **answer**: Directly answer a specific question the student has asked, if it relates to the topic. Or explain or revisit a concept the student requests explicitly.
    - **end**: Wrap up the session if the student is done or expresses no interest in continuing.
    - **continue**: The student wants to continue the current week's flow. This means moving from planning → resources → explain → quiz in order.
      - Triggered by phrases like: "continue", "go ahead", "next", "move on", "keep going", etc.
      - DO NOT infer the next step — just return "continue".
      - DO NOT use this if they say "next week" or clearly want to skip the current week.

    - **next_week**: The student wants to **skip the rest of the current week** and move on to the **next week's content**.
      - Triggered by: "next week", "skip this", "skip to week 2", "start week 2", etc.

    Now decide what action to take. Use the following format:

    Thought: <your reasoning>
    Action: <one of planning, resources, explain, quiz, answer, continue, next_week, end>
    Action Input: <input if needed>

    Feedback: {feedback}
    Topic: {state.get('topic')}
    Notes: {state.get('notes')}
    Plan: {state.get('plan')}
    Resources: {state.get('resources')}
    Phase History: {state.get('phase_history')}
    Current Week: {state.get('current_week')}
    Completed Lessons: {state.get('completed_lessons')}
    Quiz Attempts: {state.get('quiz_attempts')}
    """


In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import Tool
import os
import json
from google.colab import userdata


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite-preview-06-17",
    temperature=0.7,
    google_api_key=userdata.get("GENAI_API_KEY")
)

search_tool = TavilySearchResults(
    tavily_api_key=userdata.get("TAVILY_API_KEY"))

def generate_study_plan(state):
    prompt = build_study_plan_prompt(state['topic'], state['num_weeks'])
    output = llm.invoke(prompt).content
    print("--- Raw LLM Output for Study Plan ---")
    print(output)  # Print the raw output
    try:
        # Attempt to parse the JSON
        return json.loads(output)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print("Attempting to extract JSON from output...")
        try:
            # Look for the first '[' and last ']' to try and isolate the JSON array
            start_index = output.find('[')
            end_index = output.rfind(']')
            if start_index != -1 and end_index != -1 and end_index > start_index:
                json_string = output[start_index : end_index + 1]
                return json.loads(json_string)
            else:
                print("Could not find valid JSON array in the output.")
                return []  # Return empty plan if extraction fails
        except Exception as ex:
            print(f"Extraction attempt failed: {ex}")
            return []  # Return empty plan if extraction fai


def reset_state(state):
    state['resources'] = []
    state['quiz_attempts'] = []
    state['notes'] = []
    state['completed_lessons'] = []
    state['current_week'] = 1

In [13]:
import json
import re
def start_node(state):
    state['topic'] = input("What do you want to learn? ").strip()

    weeks_input = input("Over how many weeks would you like to study it? (press Enter to use 3 weeks): ").strip()
    try:
        num_weeks = int(weeks_input)
        if num_weeks <= 0:
            raise ValueError
    except ValueError:
        print("Using default: 3 weeks.")
        num_weeks = 3

    state['num_weeks'] = num_weeks
    state['phase_history'].append('start')
    state['phase'] = 'planning'
    print(f"--- Start Node returning. Next phase: {state['phase']} ---")
    return state

def planning_node(state):
    print("--- Study Plan ---")
    state['plan'] = generate_study_plan(state)
    # check if plan generated
    if state['plan']:
        for week in state['plan']:
            # Ensure week and focus keys exist before printing
            if 'week' in week and 'focus' in week:
                 print(f"Week {week['week']}: {week['focus']}")
            else:
                print("Warning: Invalid week structure in plan.")

        # reset if new plan generated
        reset_state(state)

        #add to history
        state['phase_history'].append('planning')
        state['phase'] = 'leader'


    else:
        print("Failed to generate a valid study plan.")
        state['phase'] = 'leader' # Return to leader to handle the failure or try again

    print(f"--- Planning Node returning. Next phase: {state['phase']} ---")
    return state


def resource_node(state):
    print("--- Finding Resources ---")
    current_week = state.get("current_week", 1)
    if not state["plan"]:
        print("No study plan found.")
        state["phase"] = "leader"
        return state

    week_info = next((week for week in state["plan"] if week["week"] == current_week), None)
    if not week_info or "focus" not in week_info:
        print(f"No valid focus found for Week {current_week}.")
        state["phase"] = "leader"
        return state

    focus = week_info["focus"]
    print(f"Resources for Week {current_week}: {focus}")
    results = search_tool.invoke(focus)  # assume returns list of {title, url}

    top_links = results[:3]
    for r in top_links:
        print(f"- {r['title']} ({r['url']})")

    # Ensure resources is a dict by week
    if not isinstance(state.get("resources"), dict):
        state["resources"] = {}

    state["resources"][current_week] = top_links

    # Phase tracking
    state['phase_history'].append('resources')
    state["phase"] = "leader"
    print(f"--- Resource Node returning. Next phase: {state['phase']} ---")
    return state


def quiz_node(state):
    import ast

    print("--- Quiz Time! ---")
    current_week = state.get("current_week", 1)
    topic = state.get("topic", "LLMs")

    week_resources = state["resources"].get(current_week, [])
    titles = [r["title"] for r in week_resources if "title" in r]

    prompt = build_quiz_generation_prompt(topic, current_week, titles)

    try:
        raw_response = llm.invoke(prompt).content
        clean_json = re.sub(r"```json|```", "", raw_response).strip()
        questions = json.loads(clean_json)
    except Exception as e:
        print("Could not generate quiz questions:", e)
        print("--- Raw LLM Output ---")
        print(raw_response)
        state["phase"] = "leader"
        return state

    correct = 0
    for idx, q in enumerate(questions):
        print(f"\nQuestion {idx + 1}: {q['question']}")
        user_answer = input("Your answer: ").strip()

        # Evaluate with LLM
        grading_prompt = build_grading_prompt(q['question'], q['answer'], user_answer)

        result = llm.invoke(grading_prompt).content
        lines = result.splitlines()

        judgment_line = next((l for l in lines if l.lower().startswith("judgment:")), "")
        explanation_line = next((l for l in lines if l.lower().startswith("explanation:")), "")

        judgment = judgment_line.split(":", 1)[1].strip().lower() if ":" in judgment_line else "unknown"
        explanation = explanation_line.split(":", 1)[1].strip() if ":" in explanation_line else result

        if judgment == "correct":
            print("Correct!")
            correct += 1
        elif judgment == "partially correct":
            print("Partially correct.")
            print(explanation)
        else:
            print("Incorrect.")
            print(explanation)

    print(f"\You got {correct} fully correct out of {len(questions)}.")
    state["quiz_attempts"].append(topic)


    state['phase_history'].append('quiz')
    state["phase"] = "leader"

    print(f"--- Quiz Node returning. Next phase: {state['phase']} ---")
    return state

def explain_node(state):
    current_week = state.get('current_week', 1)

    # Get resources for the current week
    week_resources = state['resources'].get(current_week, [])
    article_titles = [r.get('title', '') for r in week_resources if r.get('title')]

    if not article_titles:
        print("No article titles found for this week.")
        state['phase'] = 'leader'
        return state

    prompt = build_explanation_prompt(current_week, article_titles)

    explanation = llm.invoke(prompt).content

    # Show the explanation
    print("--- AI Explanation Based on Article Titles ---")
    print(explanation)

    # Log that this week was explained
    state['notes'].append(f"Week {current_week} explained based on article titles")

    # Track phase
    state['phase_history'].append('explain')

    state['phase'] = 'leader'
    print(f"--- Explain Node returning. Next phase: {state['phase']} ---")

    return state

def answer_node(state):
    question = state['user_feedback'][-1]
    topic = state['topic']
    prompt = build_answer_question_prompt(topic, question)
    ai_answer = llm.invoke(prompt).content
    print(ai_answer)
    state['phase'] = 'leader'
    print(f"--- Answer Node returning. Next phase: {state['phase']} ---")
    return state



def leader_node(state):
    # Determine the next phase hint for the prompt (for the 'continue' option)

    last_phase = state['phase_history'][-1] if state['phase_history'] else None

    if last_phase == 'planning':
        next_phase_hint = 'resources'
    elif last_phase == 'resources':
        next_phase_hint = 'explain'
    elif last_phase == 'explain':
        next_phase_hint = 'quiz'
    elif last_phase == 'quiz':
        if state['current_week'] <= state['num_weeks']:
            next_phase_hint = 'resources'
        else:
            next_phase_hint = 'end'
    else:
        next_phase_hint = 'planning'


    # 1) Collect user feedback with the next phase hint
    feedback = input(f"What would you like to do next (or any thoughts)? Type 'continue' to proceed sequentially to the {next_phase_hint} phase, or make a specific request: ")
    state['user_feedback'].append(feedback)


    prompt = build_leader_decision_prompt(state, feedback)
    decision = llm.invoke(prompt).content
    print("--- Agent Decision ---")
    print(decision)

    # 5) Parse LLM output
    action = None
    action_input = None
    # Split lines and find Action and Action Input
    for line in decision.splitlines():
        if line.lower().startswith('action:'):
            action = line.split(':',1)[1].strip().lower()
        elif line.lower().startswith('action input:'):
            action_input = line.split(':',1)[1].strip()

    # 6) Dispatch based on action
    if action == 'continue':
        # move to the next logical phase (already determined for the hint)
        if last_phase == 'quiz' and state['current_week'] < state['num_weeks']:
          state['current_week'] += 1
          print(f"Moving to week {state['current_week']}")
        state['phase'] = next_phase_hint
        print(f"--- Leader Node (Continue) returning. Next phase: {state['phase']} ---")
        return state
    if action == 'planning':
        state['phase'] = 'planning'
    elif action == 'resources':
        state['phase'] = 'resources' # Set phase to 'resources'
    elif action == 'quiz':
        state['phase'] = 'quiz' # Set phase to 'quiz'
    elif action == 'explain':
        state['phase'] = 'explain' # Set phase to 'explain'
    elif action == 'answer':
        state['phase'] = 'answer' # Set phase to 'answer'
    elif action == 'next_week':
      if state['current_week'] < len(state['plan']):
          state['current_week'] += 1
          print(f"Moving to Week {state['current_week']}")
          state['phase'] = 'resources'
      else:
          print("You've already completed all available weeks.")
          state['phase'] = 'end'
    elif action == 'end':
        state['phase'] = 'end' # Set phase to 'end'
    else:
        print(f"Unrecognized action: {action}. Returning to leader.")
        state['phase'] = 'leader' # Return to leader for unrecognized action

    print(f"--- Leader Node (Agent) returning. Next phase: {state['phase']} ---") # Print current phase

    return state


def end_node(state):
    print("Session complete. See you next time!")
    return state

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal, Dict

graph = StateGraph(CurriculumState)

# Register each node (phase) with the graph
graph.add_node('start', start_node)
graph.add_node('planning', planning_node)
graph.add_node('resources_node', resource_node)
graph.add_node('explain', explain_node)
graph.add_node('answer', answer_node)
graph.add_node('quiz', quiz_node)
graph.add_node('leader', leader_node)
graph.add_node('end', end_node)

# Set the entry point
graph.set_entry_point('start')

# Define valid transitions
graph.add_edge('start', 'planning')
graph.add_edge('planning', 'leader')
for src in ['resources_node', 'quiz', 'answer', 'explain']:
    graph.add_edge(src, 'leader')

# Use a conditional edge from leader to route based on the 'phase' in the state
# Explicitly add a transition from 'leader' to 'leader' for self-loop
graph.add_conditional_edges(
    'leader',
    # The condition is a function that takes the state and returns the next node name
    lambda state: state['phase'],
    {
        "planning": "planning",
        "resources": "resources_node", # Note: The phase name in state is 'resources', but the node name is 'resources_node'
        "quiz": "quiz",
        "answer": "answer",
        "explain": "explain",
        "leader": "leader", # Explicitly define self-loop for phase 'leader'
        "end": END # Use END for the final state
    }
)

# Final end state - already handled by conditional edge to END

# Compile into a runnable app
app = graph.compile()

# Initialize and run
initial_state = {
    'topic': '',
    'plan': [],
    'current_week': 0,
    'num_weeks': 0,
    'completed_lessons': [],
    'quiz_attempts': [],
    'notes': [],
    'resources': [],
    'user_feedback': [],
    'phase': 'start', # Initial phase
    'phase_history': [] # Initialize phase history
}

# The graph execution will start here and proceed through the defined nodes and transitions
app.invoke(initial_state)

What do you want to learn? llms
Over how many weeks would you like to study it? (press Enter to use 3 weeks): 5
--- Start Node returning. Next phase: planning ---
--- Study Plan ---
--- Raw LLM Output for Study Plan ---
[
  {
    "week": 1,
    "focus": "Foundational Concepts: What are LLMs, their history, and basic architectures (e.g., Transformers)."
  },
  {
    "week": 2,
    "focus": "LLM Training & Fine-tuning: Understanding pre-training, fine-tuning techniques, and data requirements."
  },
  {
    "week": 3,
    "focus": "LLM Applications & Capabilities: Exploring common use cases like text generation, translation, summarization, and question answering."
  },
  {
    "week": 4,
    "focus": "Prompt Engineering & Interaction: Learning how to effectively communicate with LLMs and craft optimal prompts."
  },
  {
    "week": 5,
    "focus": "Ethical Considerations & Future Trends: Discussing bias, safety, responsible deployment, and emerging research in LLMs."
  }
]
Week 1: Foundat